In [1]:
# IMPORTS
import pandas as pd 
import numpy as np 
import load_datasets


In [2]:
### SECTION 1 - DATA ANALYTICS ###
# This section looks at various statistics about the data 

In [5]:
# Import data into notebook
cafes, users, reviews = load_datasets.load_table_data()
ratings, user2cafes, cafes2users = load_datasets.create_user_review_dicts(reviews)

In [6]:
# Basics:
# Get number of users
print("Number of Users: ", len(users))
# Get number of cafes
print("Number of Cafes: ", len(cafes))
# Get number of reviews
print("Number of Reviews: ", len(reviews))

Number of Users:  196454
Number of Cafes:  15576
Number of Reviews:  1769673


In [ ]:
# Functions:
# Average rating
# Average price
# One Hot Encoding for Hours

In [ ]:
# Add more stuff...

In [ ]:
### SECTION 2 ###